## Text extraction and preparation

The text data source is from PDF documents which are mostly of conference and journal papers.

I use several tools to extract the text from PDF. One package is called **textract**

Some manual intervention (editing, and deletion) are involved as the extraction results are not ideal.

In [4]:
import os
import keras
from keras.preprocessing import text
import textract

Using TensorFlow backend.


Below is one snapshot of the folder location where a list of PDF papers which are in one category are placed into.

For this notebook, the category is for 'convolutional neural network'

In [5]:
path = 'C:/Users/k/Documents/qiqqa/324534E5-7D3E-4DAC-B4E9-057A96B7AC62'
pdf = []
for root, dr, fs in os.walk(path):
    for f in fs:
        if '.pdf' not in f:
            continue
        pdf.append(root + '/' + f)

Following, we use combination of **nltk** stopwords and RegexpTokenizer to help remove unwanted characters. Keras text preprocessing is also used to split the texts into word sequences.

In [6]:
# the snippet code from ref[1] -- see next notebook
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize

# Load stop-words
stop_words = set(stopwords.words('english'))

# Initialize tokenizer
# It's also possible to try with a stemmer or to mix a stemmer and a lemmatizer
tokenizer = RegexpTokenizer('[\'a-zA-Z]+')

In [7]:
len(pdf)

113

In [15]:
min_num_words = 5
label = 'convolutional neural network'
docs = []

In [16]:
# check if there is duplicate
dup = set()

In [17]:
for doc in pdf:
    try:
        p1 = textract.process(doc)
    except:
        print('exception..')
        pass
    p2 = str(p1)
    p3 = p2.split('\\r\\n')
    test = ''
    for i in range(0,len(p3)):
        test = test + p3[i]
    if test in dup:
        continue
    dup.add(test)
    print(p3[:5])
    print()
    i = 0
    start = False
    txt = []
    for p in p3:
        # w = [t for t in tokenizer.tokenize(p) if t.lower() not in stop_words]
        w = keras.preprocessing.text.text_to_word_sequence(p)
        if 'abstract' not in w and not start:
            continue
        if 'abstract' in w:
            start = True
            continue
        if 'references' in w:
            break
        w1 = [v for v in w if not v.lower().startswith('x')]
        if len(w1)<min_num_words:
            continue
        for word in w1:
            txt.append(word)
#         print(w1)
        i = i +1
        if i>1000:
            break
    if txt != []:
        docs.append((label,txt))

["b'arXiv:1312.6229v4 [cs.CV] 24 Feb 2014", '', 'OverFeat:', 'Integrated Recognition, Localization and Detection', 'using Convolutional Networks']

["b'SSD: Single Shot", 'MultiBox Detector', 'Wei Liu(1), Dragomir Anguelov(2), Dumitru Erhan(3), Christian Szegedy(3),', 'Scott Reed(4), Cheng-Yang Fu(1), Alexander C. Berg(1)', '']

["b'Going Deeper with Convolutions", 'Christian Szegedy1 , Wei Liu2 , Yangqing Jia1 , Pierre Sermanet1 , Scott Reed3 ,', 'Dragomir Anguelov1 , Dumitru Erhan1 , Vincent Vanhoucke1 , Andrew Rabinovich4', '1', '']

["b'Imperial College London", '', 'arXiv:1509.01549v2 [cs.AI] 14 Sep 2015', '', 'Department of Computing']

["b'REVIEW", '', 'doi:10.1038/nature14539', '', 'Deep learning']

["b'Mask R-CNN", 'Kaiming He', '', 'Georgia Gkioxari', '']

exception..
["b'Learning both Weights and Connections for Efficient", 'Neural Networks', 'Jeff Pool', 'NVIDIA', 'jpool@nvidia.com']

["b'arXiv:1512.02325v5 [cs.CV] 29 Dec 2016", '', '1', '', 'SSD: Single Shot MultiBox Detec

In [18]:
print(len(docs))

87


Save the text and category as **pkl** file

In [19]:
import _pickle as pickle

In [20]:
pickle.dump(docs, open('cnn.pkl', 'wb'))